In [1]:
import googletrans
import pandas as pd
import json 
from tqdm import tqdm

In [2]:
file_loc = "foodb_data.json"
with open(file_loc) as f:
  data = json.load(f)

In [3]:
data['1']

{'name': 'Angelica',
 'name_scientific': 'Angelica keiskei',
 'description': 'Angelica is a genus of about 60 species of tall biennial and perennial herbs in the family Apiaceae, native to temperate and subarctic regions of the Northern Hemisphere, reaching as far north as Iceland and Lapland. They grow to 1–3 m tall, with large bipinnate leaves and large compound umbels of white or greenish-white flowers. Some species can be found in purple moor and rush pastures.',
 'wikipedia_id': 'Angelica',
 'food_group': 'Herbs and Spices',
 'food_subgroup': 'Herbs',
 'orig_food': [{'orig_food_common_name': 'Angelica, fresh',
   'orig_source_name': 'Apigenin',
   'orig_content': '0.0 mg/100g'},
  {'orig_food_common_name': 'Angelica, fresh',
   'orig_source_name': 'Luteolin',
   'orig_content': '27.014285905 mg/100g'},
  {'orig_food_common_name': 'Angelica, fresh',
   'orig_source_name': 'Kaempferol',
   'orig_content': '0.0 mg/100g'},
  {'orig_food_common_name': 'Angelica, fresh',
   'orig_source

In [4]:
for key, food in data.items():
    food['orig_food'] = sorted(food['orig_food'], key=lambda x: -float(x['orig_content'].split()[0]))

- Adding important original food (with quantity) and less important (with 0 quantity but not nan)

In [5]:
for key, food in data.items():
    food['important_orig_food'] = [x['orig_source_name'] for x in food['orig_food'] if float(x['orig_content'].split()[0]) > 0.0]
    food['important_orig_quantity'] = [float(x['orig_content'].split()[0]) for x in food['orig_food'] if float(x['orig_content'].split()[0]) > 0.0]

    food['other_orig_food'] = [x['orig_source_name'] for x in food['orig_food'] if float(x['orig_content'].split()[0]) == 0.0]

- Adding compound information just for these orig_foods

In [6]:
for key, food in tqdm(data.items()): 
    food['Compounds_dict'] = {}
    for comp in food['Compound']:
        food['Compounds_dict'][comp['name']] = comp
    del food['Compound']

100%|██████████| 992/992 [00:01<00:00, 848.66it/s] 


In [7]:
for key, food in tqdm(data.items()): 
    food['important_orig_food_compounds'] = []
    for x in food['important_orig_food']:
        try:
            food['important_orig_food_compounds'].append(food['Compounds_dict'][x])
        except Exception:
            pass
    for x in food['other_orig_food']:
        try:
            food['important_orig_food_compounds'].append(food['Compounds_dict'][x])
        except Exception:
            pass

100%|██████████| 992/992 [00:00<00:00, 3239.91it/s]


- Looking for health effects in compounds

In [8]:
from collections import Counter

In [22]:
for key, food in tqdm(data.items()): 
    food['health_effects'] = []
    food['pathway_names'] = []
    for items in food['Compounds_dict'].values():
        if items['health_effect_name'] is not None:
            food['health_effects'].append(items['health_effect_name'])
        if items['pathway_name'] is not None:
            food['pathway_names'].append(items['pathway_name'])
    food['health_effects'] = sorted(dict(Counter(food['health_effects'])).items(), key=lambda x: -x[1])
    food['pathway_names'] = sorted(dict(Counter(food['pathway_names'])).items(), key=lambda x: -x[1])

100%|██████████| 992/992 [00:00<00:00, 1161.12it/s]


In [23]:
data['500']

{'name': 'Teff',
 'name_scientific': 'Eragrostis tef',
 'description': 'Eragrostis tef, teff, Williams lovegrass, annual bunch grass, taf (Tigrinya: ጣፍ? ṭaff; Amharic: ጤፍ? ṭēff) and xaafii (Oromo)), is an annual grass, a species of lovegrass native to the northern Ethiopian Highlands and Eritrean Highlands of the Horn of Africa. The word "tef" is connected by folk etymology to the Ethio-Semitic root "\\tff\\", which means "lost" (because of the small size of the grain).\r\nEragrostis tef has an attractive nutrition profile, being high in dietary fiber and iron and providing protein and calcium. It is similar to millet and quinoa in cooking, but the seed is much smaller and cooks faster, thus using less fuel.',
 'wikipedia_id': 'Teff',
 'food_group': 'Herbs and Spices',
 'food_subgroup': 'Herbs',
 'orig_food': [{'orig_food_common_name': 'Teff, cooked',
   'orig_source_name': 'Water',
   'orig_content': '74930.0 mg/100g'},
  {'orig_food_common_name': 'Teff, uncooked',
   'orig_source_nam

In [12]:
data['1'].keys()

dict_keys(['name', 'name_scientific', 'description', 'wikipedia_id', 'food_group', 'food_subgroup', 'orig_food', 'Nutrient', 'important_orig_food', 'important_orig_quantity', 'other_orig_food', 'Compounds_dict', 'important_orig_food_compounds', 'health_effects', 'pathway_names'])

In [17]:
attrs = ['name', 'name_scientific', 'description', 'food_group', 'food_subgroup', 'Nutrient', 'important_orig_food', 'important_orig_quantity', 'other_orig_food', 'important_orig_food_compounds', 'health_effects', 'pathway_names']

In [38]:
new_data = {}
for i, k in tqdm(enumerate(data.values())):
    obj = {}
    obj['name'] = k['name']
    obj['name_scientific'] = k['name_scientific']
    obj['description'] = k['description']
    obj['food_group'] = k['food_group']
    obj['food_subgroup'] = k['food_subgroup']
    obj['Nutrient'] = k['Nutrient']
    orig_food = k['important_orig_food'][:min(10, len(k['important_orig_food']))]
    orig_quantity = [f"{x} mg/100g" for x in k['important_orig_quantity'][:min(10, len(k['important_orig_quantity']))]]
    obj['main_orig_food_names'] = []
    obj['main_orig_food_quantity'] = []

    orig_found = set()
    for food, quantity in zip(orig_food, orig_quantity):
        if food in orig_found:
            continue
        obj['main_orig_food_names'].append(food)
        obj['main_orig_food_quantity'].append(quantity)
        orig_found.add(food)
    
    
    obj['other_orig_food'] = k['other_orig_food'][:min(2, len(k['other_orig_food']))]
    obj['important_compounds'] = list(set([f"{x['name']} - {x['description']}" for x in k['important_orig_food_compounds'][:min(12, len(k['important_orig_food_compounds']))]]))
    obj['health_effects'] = list(set([x[0] for x in k['health_effects'][:min(10, len(k['health_effects']))]]))
    obj['pathways'] = list(set([x[0] for x in k['pathway_names'][:min(10, len(k['pathway_names']))]]))
    new_data[i] = obj

992it [00:00, 19492.03it/s]


In [39]:
with open('refined_food_data.json', 'w') as f:
    json.dump(new_data, f, indent=2)

## Translation
- googletrans tranlates and transliterates
- Conversion of numerical text may be further required

In [10]:
from googletrans import Translator
from tqdm import tqdm

In [11]:
translator = Translator()

def Translate(eng_text):
  result = translator.translate(eng_text,src='en',dest='hi')
  return result.text

In [27]:
import json

In [28]:
with open('refined_food_data.json') as f:
    new_data = json.load(f)

In [16]:
translated_data = {}
for i, item in tqdm(new_data.items()):
    obj = {}
    
    obj['name'] = Translate(item['name'])
    try:
        obj['name_scientific'] = Translate(item['name_scientific'])
    except Exception:
        obj['name_scientific'] = ""
    try:
        obj['description'] = Translate(item['description'])
    except Exception:
        obj['description'] = ''
    try:
        obj['food_group'] = Translate(item['food_group'])
    except Exception:
        obj['food_group'] = ""
    try:
        obj['food_subgroup'] = Translate(item['food_subgroup'])
    except Exception:
        obj['food_subgroup'] = ""

    try:
        obj['main_orig_food_names'] = [Translate(x) for x in item['main_orig_food_names']]
    except Exception:
        obj['main_orig_food_names'] = []

    try:
        obj['main_orig_food_quantity'] = [Translate(x) for x in item['main_orig_food_quantity']]
    except Exception:
        obj['main_orig_food_quantity'] = []

    try:
        obj['other_orig_food'] = [Translate(x) for x in item['other_orig_food']]
    except Exception:
        obj['other_orig_food'] = []

    try:
        obj['important_compounds'] = [Translate(x) for x in item['important_compounds']]
    except Exception:
        obj['important_compounds'] = []

    try:
        obj['health_effects'] = [Translate(x) for x in item['health_effects']]
    except Exception:
        obj['health_effects'] = []

    try:
        obj['pathways'] = [Translate(x) for x in item['pathways']]
    except Exception:
        obj['pathways'] = []

    translated_data[i] = obj

100%|██████████| 992/992 [1:46:55<00:00,  6.47s/it]  


In [17]:
with open("translated_food_data.json", 'w') as f:
    json.dump(translated_data, f, indent=2)

In [58]:
with open("viewable_food_hindi_data.json", 'r') as f:
    data = json.load(f)

In [30]:
data['1']

{'name': 'सेवॉय गोभी',
 'name_scientific': 'ब्रैसिका ओलेरासिया संस्करण। सबौदा',
 'description': 'सेवॉय गोभी (ब्रासिका ओलेरासिया कॉनवर। कैपिटाटा वर। सबौडा एल।) गोभी की एक किस्म है, जो पौधे की प्रजाति ब्रैसिका ओलेरासिया की एक किस्म है। सेवॉय गोभी सर्दियों की सब्जी है। सेवॉय गोभी की एक किस्म जनवरी किंग गोभी है। सेवॉय गोभी का इस्तेमाल कई तरह के व्यंजनों में किया जा सकता है। यह रेड वाइन, सेब, मसाले, सहिजन और मांस के साथ अच्छी तरह से जुड़ जाता है। इसका उपयोग रौलेड्स के लिए, स्टॉज और सूप में, साथ ही भुना हुआ सादा और जैतून के तेल के साथ बूंदा बांदी के लिए किया जा सकता है। पत्तागोभी जो अपने आकार के लिए भारी होती है, जिसमें बेदाग और चमकदार, ताजा दिखने वाली पत्तियां वांछनीय गुणवत्ता के संकेत हैं। जब भी संभव हो साबुत गोभी को प्राथमिकता दी जाती है क्योंकि पहले से कटी हुई या पहले से कटी हुई गोभी में विटामिन की मात्रा बहुत कम होती है। अधिकांश गोभी के लिए पीक सीजन नवंबर से अप्रैल तक चलता है। ताजा साबुत पत्तागोभी प्रकार और किस्म के आधार पर एक से छह सप्ताह तक रेफ्रिजरेटर में रखेगी। सख्त हरी, सफेद या लाल

In [55]:
hindi_digit_map = {
    '0': '०',
    '1': '१',
    '2': '२',
    '3' :'३',
    '4' :'४',
    '5' :'५',
    '6' :'६',
    '7' :'७',
    '8' :'८',
    '9' :'९'
}

In [58]:
for idx, food in data.items():
    for eng_num, hin_num in hindi_digit_map.items():
        food['description'] = food['description'].replace(eng_num, hin_num)
        food['main_orig_food_quantity'] = [x.replace(eng_num, hin_num) for x in food['main_orig_food_quantity']]
        food['important_compounds'] = [x.replace(eng_num, hin_num) for x in food['important_compounds']]
        food['other_orig_food'] = [x.replace(eng_num, hin_num) for x in list(set(food['other_orig_food']))]
    food['img_url'] = f"https://foodb.ca/system/foods/pictures/{int(idx)+1}/full/{int(idx)+1}.png"

data['1']

{'name': 'सेवॉय गोभी',
 'name_scientific': 'ब्रैसिका ओलेरासिया संस्करण। सबौदा',
 'description': 'सेवॉय गोभी (ब्रासिका ओलेरासिया कॉनवर। कैपिटाटा वर। सबौडा एल।) गोभी की एक किस्म है, जो पौधे की प्रजाति ब्रैसिका ओलेरासिया की एक किस्म है। सेवॉय गोभी सर्दियों की सब्जी है। सेवॉय गोभी की एक किस्म जनवरी किंग गोभी है। सेवॉय गोभी का इस्तेमाल कई तरह के व्यंजनों में किया जा सकता है। यह रेड वाइन, सेब, मसाले, सहिजन और मांस के साथ अच्छी तरह से जुड़ जाता है। इसका उपयोग रौलेड्स के लिए, स्टॉज और सूप में, साथ ही भुना हुआ सादा और जैतून के तेल के साथ बूंदा बांदी के लिए किया जा सकता है। पत्तागोभी जो अपने आकार के लिए भारी होती है, जिसमें बेदाग और चमकदार, ताजा दिखने वाली पत्तियां वांछनीय गुणवत्ता के संकेत हैं। जब भी संभव हो साबुत गोभी को प्राथमिकता दी जाती है क्योंकि पहले से कटी हुई या पहले से कटी हुई गोभी में विटामिन की मात्रा बहुत कम होती है। अधिकांश गोभी के लिए पीक सीजन नवंबर से अप्रैल तक चलता है। ताजा साबुत पत्तागोभी प्रकार और किस्म के आधार पर एक से छह सप्ताह तक रेफ्रिजरेटर में रखेगी। सख्त हरी, सफेद या लाल

In [59]:
with open('finetuned_hindi_data.json', 'w') as f:
    json.dump(data, f, indent=2)

In [59]:
with open('foodb_data.json') as f:
    updated_data = json.load(f)

In [60]:
updated_data['2']

{'foodb_id': '2',
 'name': 'Savoy cabbage',
 'name_scientific': 'Brassica oleracea var. sabauda',
 'description': 'Savoy cabbage (Brassica oleracea convar. capitata var. sabauda L. ) is a variety of the cabbage, a cultivar of the plant species Brassica oleracea. Savoy cabbage is a winter vegetable. A variety of the savoy cabbage is the January King Cabbage. Savoy cabbage can be used in a variety of recipes. It pairs well with red wine, apples, spices, horseradish and meat. It can be used for roulades, in stews and soups, as well as roasted plain and drizzled with olive oil. Cabbage that is heavy for its size with leaves that are unblemished and have a bright, fresh look are signs of desirable quality. Whole cabbages are preferred whenever possible as pre-cut or preshredded cabbage has a greatly diminished vitamin content. Peak season for most cabbages runs from November through April. Fresh whole cabbage will keep in the refrigerator for one to six weeks depending on type and variety. 

In [61]:
reordered_new_data = {}
for i, k in enumerate(updated_data.values()):
    reordered_new_data[i] = k

In [62]:
for key, val in reordered_new_data.items():
    data[str(key)]['foodb_id'] = val['foodb_id']
    data[str(key)]['nutritional_info'] = val['nutritional_info']
    data[str(key)]['img_url'] = f"https://foodb.ca/system/foods/pictures/{val['foodb_id']}/full/{val['foodb_id']}.png"

In [63]:
data['1']

{'name': 'सेवॉय गोभी',
 'name_scientific': 'ब्रैसिका ओलेरासिया संस्करण। सबौदा',
 'description': 'सेवॉय गोभी (ब्रासिका ओलेरासिया कॉनवर। कैपिटाटा वर। सबौडा एल।) गोभी की एक किस्म है, जो पौधे की प्रजाति ब्रैसिका ओलेरासिया की एक किस्म है। सेवॉय गोभी सर्दियों की सब्जी है। सेवॉय गोभी की एक किस्म जनवरी किंग गोभी है। सेवॉय गोभी का इस्तेमाल कई तरह के व्यंजनों में किया जा सकता है। यह रेड वाइन, सेब, मसाले, सहिजन और मांस के साथ अच्छी तरह से जुड़ जाता है। इसका उपयोग रौलेड्स के लिए, स्टॉज और सूप में, साथ ही भुना हुआ सादा और जैतून के तेल के साथ बूंदा बांदी के लिए किया जा सकता है। पत्तागोभी जो अपने आकार के लिए भारी होती है, जिसमें बेदाग और चमकदार, ताजा दिखने वाली पत्तियां वांछनीय गुणवत्ता के संकेत हैं। जब भी संभव हो साबुत गोभी को प्राथमिकता दी जाती है क्योंकि पहले से कटी हुई या पहले से कटी हुई गोभी में विटामिन की मात्रा बहुत कम होती है। अधिकांश गोभी के लिए पीक सीजन नवंबर से अप्रैल तक चलता है। ताजा साबुत पत्तागोभी प्रकार और किस्म के आधार पर एक से छह सप्ताह तक रेफ्रिजरेटर में रखेगी। सख्त हरी, सफेद या लाल

In [64]:
for key, val in data.items():
    for k in val['nutritional_info']:
        try:
            val['nutritional_info'][k] = f"{(float(val['nutritional_info'][k])/1000):.4f}"
        except Exception:
            pass

In [65]:
with open('finetuned_hindi_data.json', 'w') as f:
    json.dump(data, f, indent=2)

In [66]:
data['1']

{'name': 'सेवॉय गोभी',
 'name_scientific': 'ब्रैसिका ओलेरासिया संस्करण। सबौदा',
 'description': 'सेवॉय गोभी (ब्रासिका ओलेरासिया कॉनवर। कैपिटाटा वर। सबौडा एल।) गोभी की एक किस्म है, जो पौधे की प्रजाति ब्रैसिका ओलेरासिया की एक किस्म है। सेवॉय गोभी सर्दियों की सब्जी है। सेवॉय गोभी की एक किस्म जनवरी किंग गोभी है। सेवॉय गोभी का इस्तेमाल कई तरह के व्यंजनों में किया जा सकता है। यह रेड वाइन, सेब, मसाले, सहिजन और मांस के साथ अच्छी तरह से जुड़ जाता है। इसका उपयोग रौलेड्स के लिए, स्टॉज और सूप में, साथ ही भुना हुआ सादा और जैतून के तेल के साथ बूंदा बांदी के लिए किया जा सकता है। पत्तागोभी जो अपने आकार के लिए भारी होती है, जिसमें बेदाग और चमकदार, ताजा दिखने वाली पत्तियां वांछनीय गुणवत्ता के संकेत हैं। जब भी संभव हो साबुत गोभी को प्राथमिकता दी जाती है क्योंकि पहले से कटी हुई या पहले से कटी हुई गोभी में विटामिन की मात्रा बहुत कम होती है। अधिकांश गोभी के लिए पीक सीजन नवंबर से अप्रैल तक चलता है। ताजा साबुत पत्तागोभी प्रकार और किस्म के आधार पर एक से छह सप्ताह तक रेफ्रिजरेटर में रखेगी। सख्त हरी, सफेद या लाल